# Bài thực hành số 3:  

Chào mừng đến với bài thực hành số 3, trong bài thực hành này ta sẽ học cách áp dụng thuật toán Canny vào việc nhận diện làn đường cho xe tự lái.

# Tổng quan

Xe tự lái là một trong những cải tiến mang tính đột phá nhất trong lĩnh vực AI. Một trong nhiều bước liên quan đến quá trình đào tạo xe tự lái là phát hiện làn đường, đây là bước sơ bộ. Trong bài thực hành này, chúng ta sẽ tìm hiểu cách thực hiện phát hiện làn đường bằng thuật toán Canny.

# Mục tiêu học tập

Sau khi hoàn thành bài thực hành này, học viên sẽ học được các kiến thức:
- Các kĩ thuật xử lí ảnh thời gian thực.
- Áp dụng Canny để nhận diện làn đường.

# Những kiến thức liên quan

- Kĩ năng lập trình cơ bản với Python
- Sử dụng được các hàm cơ bản của OpenCV
- Thuật toán Canny.

# Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể lập trình và chạy được xe bám làn.


# Bài toán

**Mục tiêu**: Mô hình xe có thể chạy bám làn đường với tốc độ chậm.

**Yêu cầu**: Xử lý nhận diện làn đường bằng thuật toán Canny.

## Thư viện

Đầu tiên, để bắt đầu, ta cần nhập các thư viện cần thiết.

In [1]:
import cv2
import numpy as np
import glob
import matplotlib.image as mpimage
import requests
import time
from urllib.request import urlopen
from queue import Queue
import threading

## Calibration

Calibration trong xử lý ảnh là quá trình điều chỉnh các cảm biến và hệ thống quang học của máy ảnh để đảm bảo rằng ảnh được ghi lại chính xác phản ánh thực tế.

Hàm 'CameraCalibration' sẽ có nhiệm vụ chỉnh **Độ méo ống kính** bằng cách sử dụng các ảnh của bàn cờ vua. Bàn cờ vua được sử dụng vì chúng có các góc dễ dàng nhận dạng bởi thuật toán.

1. **Khởi tạo:**:
    - Khởi tạo các danh sách và mảng cần thiết.

2. **Xử lý từng ảnh:**:
    - Sử dụng hàm 'cv2.findChessboardCorners' để tìm các góc bàn cờ trong ảnh.

3. **Hiệu chuẩn camera:**
    - Sử dụng hàm 'cv2.calibrateCamera' của OpenCV để tính toán các tham số nội tại của camera (mtx, dist) dựa trên các danh sách 'objpoints' và 'imgpoints'.

4. **Hàm 'undistort':**
    - Hàm này cho phép người dùng loại bỏ biến dạng thấu kính (distortion) từ ảnh bằng cách sử dụng các tham số nội tại được tính toán (mtx, dist) và hàm 'cv2.undistort' của OpenCV.





In [2]:
class CameraCalibration():
    """ Class that calibrate camera using chessboard images """

    def __init__(self, image_dir, nx, ny):
        """ Init CameraCalibration.

        Parameters:
            image_dir (str): path to folder contains chessboard images
            nx (int): width of chessboard (number of squares)
            ny (int): height of chessboard (number of squares)
        """
        # Using glob to get a list of all image filenames in the specified directory
        fnames = glob.glob("{}/*".format(image_dir))

        objpoints = []  # List to store 3D points of chessboard corners
        imagepoints = []  # List to store 2D points of chessboard corners in the image

        # Coordinates of chessboard's corners in 3D
        # Generating coordinates for 3D points for chessboard corners
        objp = np.zeros((nx*ny, 3), np.float32)
        # Filling the array with coordinates of the chessboard corners
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

        # Go through all chessboard images
        for f in fnames:
            # Loading image using matplotlib's imread function
            image = mpimage.imread(f)

            # Using OpenCV's function to find chessboard corners
            ret, corners = cv2.findChessboardCorners(image, (nx, ny))
            # If corners are found
            if ret:
                # Append the corner coordinates to the list
                imagepoints.append(corners)
                # Append the 3D coordinates to the list
                objpoints.append(objp)

        # Get shape of the last image processed
        shape = (image.shape[1], image.shape[0])
        ret, self.mtx, self.dist, _, _ = cv2.calibrateCamera(
            # Calibrating the camera using OpenCV's calibrateCamera function
            objpoints, imagepoints, shape, None, None)

        # If calibration was unsuccessful
        if not ret:
            raise Exception("Unable to calibrate camera")

    # Undistort an image
    def undistort(self, image):
        """ Return undistort image.

        Parameters:
            image (np.array): Input image

        Returns:
            Image (np.array): Undistorted image
        """
        # Undistort the grayscale image using OpenCV's undistort function
        return cv2.undistort(image, self.mtx, self.dist, None, self.mtx)

**Bài tập 1:** Hãy hoàn thành đoạn code sau bằng cách điền vào [...].

Biến dang quang học là hiện tượng phổ biến khi chụp ảnh. Sự biến dạng xuyên tâm làm cho các đường thẳng có vẻ cong. Biến dạng xuyên tâm trở nên lớn hơn khi các điểm ở xa tâm ảnh hơn. 
Ví dụ: một hình ảnh được bên dưới cho thấy hai cạnh của bàn cờ (đánh dấu bằng các đường màu đỏ). 


<img src="Markdown Image\Chessboard1.png" alt="" width="640" height="360">


Tuy nhiên, bạn có thể thấy đường viền của bàn cờ không phải là một đường thẳng và không khớp với đường màu đỏ.


Để giải quyết vấn đề này chúng ta cần Camera Calibration (Hiệu chuẩn máy ảnh)


<img src="Markdown Image\Chessboard2.png" alt="" width="640" height="360">

Dữ liệu đầu cho việc hiệu chỉnh camera là tập hợp các điểm trong thế giới thực 3D và tọa độ 2D tương ứng của các điểm này trong ảnh.
Trong bài lap này chúng ta sẽ xác định các điểm giao nhau trong hình ảnh bàn cờ




In [ ]:
# Đường dẫn đến thư mục chứa các ảnh bàn cờ.
image_dir = r'[...]'
nx = [...]  # Số điểm giao nhau trong 1 hàng
ny = [...]  # Số điểm giao nhau trong 1 cột

# Initialize camera calibration object
calibrator = CameraCalibration([...], [...], [...])

# Kiểm tra kết quả
# Đường dẫn đến 1 ảnh bàn cờ
example_image = cv2.imread(r'[...]')
# Undistort the example image
undistorted_image = calibrator.undistort(example_image)
# Show the original and undistorted image (optional)
cv2.imshow("Original Image", example_image)
cv2.imshow("Undistorted Image", undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
image_dir = r'E:/Code/ACE/AI/LaneDetection/camera_cal'
nx = 9
ny = 6

# Initialize camera calibration object
calibrator = CameraCalibration(image_dir, nx, ny)

# Kiểm tra kết quả
# Đường dẫn đến 1 ảnh bàn cờ
example_image = cv2.imread(
    r'E:/Code/ACE\AI/LaneDetection/Lap/Markdown Image/Chessboard.jpg')
# Undistort the example image
undistorted_image = calibrator.undistort(example_image)
# Show the original and undistorted image (optional)
cv2.imshow("Original Image", example_image)
cv2.imshow("Undistorted Image", undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Chuyển góc nhìn ảnh

Lớp 'PerspectiveTransformation' dùng để chuyển đổi ảnh giữa góc nhìn phía trước và góc nhìn trên xuống (up to down).

1. **Khởi tạo**:
    - Thiết lập các điểm nguồn và điểm đích cho phép biến đổi.
    - Điểm nguồn là bốn điểm trên ảnh gốc tượng trưng cho góc nhìn phía trước. - Điểm đích là bốn điểm tạo thành một hình chữ nhật xác định vùng nhìn trên xuống.

2. **Hàm 'forward'**:
    - Thực hiện phép biến đổi để chuyển ảnh từ góc nhìn phía trước thành góc nhìn trên xuống.

3. **Hàm 'backward'**:
    -  Thực hiện phép biến đổi ngược lại, chuyển ảnh từ góc nhìn trên xuống về góc nhìn phía trước.

In [12]:
class PerspectiveTransformation:
    """ Transforming image between front view and top view """

    def __init__(self, src, dst):
        # Compute perspective transformation matrix
        self.M = cv2.getPerspectiveTransform(src, dst)
        # Compute inverse perspective transformation matrix
        self.M_inv = cv2.getPerspectiveTransform(dst, src)

    # Convert image to bird-eye view
    def forward(self, image, image_size=(1280, 720), flags=cv2.INTER_LINEAR):
        """ Take a front view image and transform to top view

        Parameters:
            image (np.array): A front view image
            image_size (tuple): Size of the image (width, height)
            flags : flag to use in cv2.warpPerspective()

        Returns:
            Image (np.array): Top view image
        """
        return cv2.warpPerspective(image, self.M, image_size, flags=flags)

    # Restore image from bird-eye view to original perspective
    def backward(self, image, image_size=(1280, 720), flags=cv2.INTER_LINEAR):
        """ Take a top view image and transform it to front view

        Parameters:
            image (np.array): A top view image
            image_size (tuple): Size of the image (width, height)
            flags (int): flag to use in cv2.warpPerspective()

        Returns:
            Image (np.array): Front view image
        """
        return cv2.warpPerspective(image, self.M_inv, image_size, flags=flags)

**Bài tập 2:** Hãy hoàn thành đoạn code sau bằng cách điền vào [...].

Trong việc điều khiển xe tự hành, việc xác định làn đường là vô cùng quan trọng. 
Để phát hiện được đường đi một cách dễ dàng và hiệu quả hơn, chúng ta có thể sử dụng phương pháp chuyển đổi góc nhìn giữa góc nhìn trước và góc nhìn từ trên xuống (bird's-eye view).

<img src="Markdown Image\Birds-eye_view.png" alt="" width="640" height="200">
                                            
Hàm forward của class PerspectiveTransformation sẽ chuyển các điểm gốc (src_points) sang các điểm đích (dst_points) để chuyển góc nhìn của ảnh. Các điểm là tọa độ của pixel trên ảnh đầu vào. 

<img src="Markdown Image\xy_aris.png" alt="" width="300" height="300">

Điểm top-left và bottom-left của src-point sẽ trùng với vạch kẻ đường khi lốp xe bên phải của bộ xe ACE-kit chạm vào vạch đường bên phải và tương tự với bên còn lại.
Vì hạn chế của phần cứng nên ta nên lấy 1/3 ảnh dưới (y = 450) để việc xác đinh đường đi (ỏ phần sau) có độ ổn định cao                                                                                                                       



In [ ]:
# Đường dẫn đến 1 ảnh đường đi
example_image = cv2.imread("[...]")
example_image = cv2.resize(example_image, (1280, 720))

# Define source points for perspective transformation
src_points = np.float32([([...], [...]),      # top-left
                        ([...], [...]),       # bottom-left
                        ([...], [...]),     # bottom-right
                        ([...], [...])])     # top-right

# Define destination points for perspective transformation
dst_points = np.float32([(100, 0),
                        (100, 720),
                        (1180, 720),
                        (1180, 0)])

# Draw point in image
image = example_image
# Draw destination points (in blue)
for i, point in enumerate(src_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (255, 0, 0), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Draw destination points (in red)
for i, point in enumerate(dst_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (0, 0, 255), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
cv2.imshow("Normal Image", image)

# Transform image to bird-eye view
transform = PerspectiveTransformation([...], [...])
transform_image = transform.forward([...])

# Show the transform image
cv2.imshow("Transformed Image", transform_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
# Đường dẫn đến 1 ảnh đường đi
example_image = cv2.imread(r'E:/Code/ACE/AI/LaneDetection/Lap/6.jpg')
example_image = cv2.resize(example_image, (1280, 720))

# Define source points for perspective transformation
src_points = np.float32([(550, 450),      # top-left
                        (80, 720),       # bottom-left
                        (1200, 720),     # bottom-right
                        (730, 450)])     # top-right

# Define destination points for perspective transformation
dst_points = np.float32([(100, 0),
                        (100, 720),
                        (1180, 720),
                        (1180, 0)])

# Draw point in image
image = example_image.copy()
# Draw destination points (in blue)
for i, point in enumerate(src_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (255, 0, 0), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Draw destination points (in red)
for i, point in enumerate(dst_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (0, 0, 255), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
cv2.imshow("Normal Image", image)

# Transform image to bird-eye view
transform = PerspectiveTransformation(src_points, dst_points)
transform_image = transform.forward(example_image)

# Show the transform image
cv2.imshow("Transformed Image", transform_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Thresholding

Class 'Thresholding' thực hiện phân ngưỡng ảnh để tách ra các vạch kẻ làn đường trong ảnh. Hàm sử dụng hai phương pháp phân ngưỡng: phân ngưỡng tuyệt đối và phân ngưỡng tương đối.
    
H (Hue): biểu diễn loại màu sắc trong vòng tròn quang phổ với đơn vị đo là góc có giá trị thuộc khoảng  [0,179].

S (Saturation): biểu diễn mức độ bão hòa màu với mức độ thuộc khoảng [0,255].

V (hay B) (Value hay Bright): biểu diễn độ sáng của màu với dải giá trị thuộc khoảng [0,255].   


1. **Phân ngưỡng tuyệt đối**: 'threshold_abs()'
   - được sử dụng trên kênh H (Hue) của ảnh HLS để trích xuất vùng màu của vạch kẻ làn đường.
   - Giá trị tham số low và high là ngưỡng giá trị màu trong thang HUE.
      + 20, 30 là màu vàng

2. **Phân ngưỡng tương đối**: 'threshold_rel()'
   - được sử dụng trên kênh V (Value) của ảnh HSV để trích xuất các vùng sáng tối của ảnh, thông qua mức độ tương phản.


Hàm thực hiện phân ngưỡng cho cả làn đường bên trái và làn đường bên phải, sau đó kết hợp kết quả phân ngưỡng của hai làn đường lại với nhau để tạo thành ảnh nhị phân cuối cùng, chỉ chứa các pixel thuộc vạch kẻ làn đường.

In [31]:
# Calculate relative thresholding
def threshold_rel(image, lo, hi):
    # Calculate the minimum pixel value in the image
    vmin = np.min(image)
    # Calculate the maximum pixel value in the image
    vmax = np.max(image)

    # Compute the low threshold value based on the given percentage 'lo'
    vlo = vmin + (vmax - vmin) * lo
    # Compute the high threshold value based on the given percentage 'hi'
    vhi = vmin + (vmax - vmin) * hi
    # Apply the thresholding operation and convert to uint8 data type
    return np.uint8((image >= vlo) & (image <= vhi)) * 255


def threshold_abs(image, lo, hi):  # Function definition for absolute thresholding
    # Apply the thresholding operation and convert to uint8 data type
    return np.uint8((image >= lo) & (image <= hi)) * 255


class Thresholding:
    """ Extracting relevant pixels in an image """

    def __init__(self):  # Constructor method for the class (currently empty)
        pass

    def forward(self, image):
        """ Take an image and extract all relavant pixels.

        Parameters:
            image (np.array): Input image

        Returns:
            binary (np.array): A binary image represent all positions of relavant pixels.
        """
        # Convert the input image to HLS color space
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        # Convert the input image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        # Extract the H (hue) channel from the HLS image
        h_channel = hls[:, :, 0]
        # Extract the S (saturation) channel from the HLS image
        s_channel = hls[:, :, 2]
        # Extract the V (value) channel from the HSV image
        v_channel = hsv[:, :, 2]

        # Check if left lane line is  yellow
        # left_lane = threshold_abs(h_channel, 20, 30)

        # Get left lane line pixel
        left_lane = threshold_rel(v_channel, 0.7, 1.0)

        # Get right lane line pixel
        right_lane = threshold_rel(v_channel, 0.7, 1.0)

        # Combine the thresholded images for left and right lanes using logical OR
        result = left_lane | right_lane

        return result  # Return the resulting binary image with lane markings

**Bài tập 3:** Hãy hoàn thành đoạn code sau bằng cách điền vào [...].                                  

In [ ]:
# Đường dẫn đến 1 ảnh đường đi
example_image = cv2.imread(r'[...]')
example_image = cv2.resize(example_image, (1280, 720))

thresholding = Thresholding()

image = example_image.copy()
undistort_image = calibrator.undistort([...])
bird_eye_image = transform.forward([...])
thresholding_image = thresholding.forward([...])


# Show the transform image
cv2.imshow("Thresholding Image", [...])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
# Đường dẫn đến 1 ảnh đường đi
example_image = cv2.imread(
    r'E:\Code\ACE\AI\LaneDetection\Lap\Test Image\Lane_line.jpg')
example_image = cv2.resize(example_image, (1280, 720))


thresholding = Thresholding()

image = example_image.copy()
undistort_image = calibrator.undistort(image)
bird_eye_image = transform.forward(undistort_image)
thresholding_image = thresholding.forward(bird_eye_image)


# Show the transform image
cv2.imshow("Normal Image", example_image)
cv2.imshow("Transformed Image", thresholding_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Nhận diện làn đường

Hàm này dùng để phát hiện vạch kẻ làn đường trong ảnh. Nó là một lớp có tên là 'LaneLines'.

1. **Khởi tạo**:
    - Lớp tạo ra các biến để lưu trữ các thông tin về vạch kẻ làn đường và các thuộc tính của lớp.

2. **Hàm 'hist()'**:
    - Tính toán histogram của nửa dưới của ảnh.

3. **Hàm 'pixels_in_window()'**:
    - Trả về tất cả các điểm ảnh nằm trong một cửa sổ cụ thể.

4. **Hàm 'extract_features()'**:
    - Trích xuất các đặc trưng từ ảnh nhị phân, lưu trữ kích thước cửa sổ trượt, các chỉ số của điểm ảnh khác 0 trong ảnh và lưu trữ tọa độ x, y của các điểm ảnh đó.

5. **Hàm 'find_lane_pixels()'**:
    - Tìm các điểm ảnh thuộc vạch kẻ làn đường trong một ảnh nhị phân đã được biến dạng.
    - Trả về tọa độ x, y của các điểm ảnh vạch kẻ cho mỗi bên trái và phải, cùng với ảnh RGB được sử dụng để hiển thị kết quả sau này.

6. **Hàm 'fit_poly()'**:
    - Tính toán đường cong cho các điểm ảnh vạch kẻ được phát hiện và vẽ lên ảnh.

7. **Hàm 'plot()'**:
    - Thêm thông tin bổ sung vào ảnh đầu ra, bao gồm hướng di chuyển và độ cong của vạch kẻ làn đường.

8. **Hàm 'measure_curvature()'**:
    - Đo độ cong của vạch kẻ làn đường và vị trí xe.

9. **Hàm 'forward()'**:
    - Xử lý ảnh đầu vào và phát hiện vạch kẻ làn đường.
    - Trả về ảnh RGB với các điểm ảnh vạch kẻ được đánh dấu và các chi tiết khác.

In [ ]:
# Compute histogram of bottom half of the image
def hist(image):
    # Select bottom half of the image
    bottom_half = image[image.shape[0]//2:, :]
    # Compute histogram along the horizontal axis
    return np.sum(bottom_half, axis=0)


# Class for lane line detection
class LaneLines:
    """ Detected lane lines """

    def __init__(self):
        self.left_fit = None    # Coefficients of the left lane polynomial fit
        self.right_fit = None   # Coefficients of the right lane polynomial fit
        self.nonzero = None     # Indices of nonzero pixels in the image
        self.nonzerox = None    # X coordinates of nonzero pixels
        self.nonzeroy = None    # Y coordinates of nonzero pixels
        self.dir = []           # List to store directions of detected lanes
        self.msg = "Straight"   # Default message for lane direction

        self.nwindows = 9   # Number of sliding windows for lane detection
        self.margin = 30    # Margin around the previous window to search for lane pixels
        self.minpix = 50    # Minimum number of pixels to recenter the window

    # Method to compute pixel indices within a window

    def pixels_in_window(self, center, margin, height):
        """ Return all pixel that in a specific window

        Parameters:
            center (tuple): coordinate of the center of the window
            margin (int): half width of the window
            height (int): height of the window

        Returns:
            pixelx (np.array): x coordinates of pixels that lie inside the window
            pixely (np.array): y coordinates of pixels that lie inside the window
        """
        # Top-left corner of the window
        topleft = (center[0]-margin, center[1]-height//2)
        # Bottom-right corner of the window
        bottomright = (center[0]+margin, center[1]+height//2)

        # Condition to select pixels within the window boundaries
        condx = (topleft[0] <= self.nonzerox) & (
            self.nonzerox <= bottomright[0])
        condy = (topleft[1] <= self.nonzeroy) & (
            self.nonzeroy <= bottomright[1])
        # Return selected pixel coordinates
        return self.nonzerox[condx & condy], self.nonzeroy[condx & condy]

    # Method to extract features from the input image
    def extract_features(self, image):
        """ Extract features from a binary image

        Parameters:
            image (np.array): A binary image
        """
        self.image = image  # Store the input image
        # Compute the height of each sliding window
        self.window_height = int(image.shape[0]//self.nwindows)

        # Compute indices of nonzero pixels in the image
        self.nonzero = image.nonzero()
        self.nonzerox = np.array(self.nonzero[1])
        self.nonzeroy = np.array(self.nonzero[0])

    # Method to find lane pixels within sliding windows
    def find_lane_pixels(self, image):
        """Find lane pixels from a binary warped image.

        Parameters:
            image (np.array): A binary warped image

        Returns:
            leftx (np.array): x coordinates of left lane pixels
            lefty (np.array): y coordinates of left lane pixels
            rightx (np.array): x coordinates of right lane pixels
            righty (np.array): y coordinates of right lane pixels
            out_image (np.array): A RGB image that use to display result later on.
        """
        # Ensure input image is grayscale
        assert (len(image.shape) == 2)
        # Create an image to draw detected lane pixels
        out_image = np.dstack((image, image, image))

        # Compute histogram of the bottom half of the image
        histogram = hist(image)
        # Compute midpoint of the histogram
        midpoint = histogram.shape[0]//2
        # Find the peak of the left half of the histogram
        leftx_base = np.argmax(histogram[:midpoint])
        # Find the peak of the right half of the histogram
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Initialize current x coordinate for left lane
        leftx_current = leftx_base
        # Initialize current x coordinate for right lane
        rightx_current = rightx_base
        # Initialize current y coordinate
        y_current = image.shape[0] + self.window_height//2
        # Initialize lists to store detected lane pixels
        leftx, lefty, rightx, righty = [], [], [], []

        # Loop through each sliding window
        for _ in range(self.nwindows):
            # Move the window position upward
            y_current -= self.window_height

            # Define center coordinates of the current sliding window
            center_left = (leftx_current, y_current)
            center_right = (rightx_current, y_current)

            # Find lane pixels within the current sliding window
            good_left_x, good_left_y = self.pixels_in_window(
                center_left, self.margin, self.window_height)
            good_right_x, good_right_y = self.pixels_in_window(
                center_right, self.margin, self.window_height)

            # Append detected lane pixels to the respective lists
            leftx.extend(good_left_x)
            lefty.extend(good_left_y)
            rightx.extend(good_right_x)
            righty.extend(good_right_y)

            # Recenter the sliding window if enough pixels are found
            if len(good_left_x) > self.minpix:
                leftx_current = np.int32(np.mean(good_left_x))
            if len(good_right_x) > self.minpix:
                rightx_current = np.int32(np.mean(good_right_x))

        # Return detected lane pixels and the image with visualization
        return leftx, lefty, rightx, righty, out_image

    # Fit polynomial curves to the detected lane pixels
    def fit_poly(self, image):
        """Find the lane line from an image and draw it.

        Parameters:
            image (np.array): a binary warped image

        Returns:
            out_image (np.array): a RGB image that have lane line drawn on that.
        """
        # Find lane pixels''
        leftx, lefty, rightx, righty, out_image = self.find_lane_pixels(
            image)
        # Fit polynomial curves to the detected lane pixels
        if len(lefty) > 1500:
            self.left_fit = np.polyfit(lefty, leftx, 2)
        if len(righty) > 1500:
            self.right_fit = np.polyfit(righty, rightx, 2)

        # Compute y coordinates for plotting the lane lines
        maxy = image.shape[0] - 1
        miny = image.shape[0] // 3
        if len(lefty):
            maxy = max(maxy, np.max(lefty))
            miny = min(miny, np.min(lefty))
        if len(righty):
            maxy = max(maxy, np.max(righty))
            miny = min(miny, np.min(righty))
        ploty = np.linspace(miny, maxy, image.shape[0])

        # Compute x coordinates for plotting the lane lines
        left_fitx = self.left_fit[0]*ploty**2 + \
            self.left_fit[1]*ploty + self.left_fit[2]
        right_fitx = self.right_fit[0]*ploty**2 + \
            self.right_fit[1]*ploty + self.right_fit[2]

        # Draw the lane lines on the output image
        for i, y in enumerate(ploty):
            l = int(left_fitx[i])
            r = int(right_fitx[i])
            y = int(y)
            cv2.line(out_image, (l, y), (r, y), (0, 255, 0))

        # Measure curvature and return the image with lane lines drawn
        lR, rR, pos = self.measure_curvature()
        return out_image

    # Method to plot additional information on the output image
    def plot(self, out_image):
        np.set_printoptions(precision=6, suppress=True)
        lR, rR, pos = self.measure_curvature()  # Measure curvature

        # Determine driving direction based on lane curvature
        value = None
        if abs(self.left_fit[0]) > abs(self.right_fit[0]):
            value = self.left_fit[0]
        else:
            value = self.right_fit[0]

        # Update driving direction history
        if abs(value) <= 0.00020:
            self.dir.append('F')  # Forward
        elif value < 0:
            self.dir.append('L')  # Left
        else:
            self.dir.append('R')  # Right

        # Maintain a history of driving directions
        if len(self.dir) > 10:
            self.dir.pop(0)

        # Determine the predominant driving direction
        direction = max(set(self.dir), key=self.dir.count)

        # Display lane direction and curvature information on the output image
        curvature_msg = "Curvature = {:.0f} m".format(min(lR, rR))
        if direction == 'L':
            self.msg = "Left"
        if direction == 'R':
            self.msg = "Right"
        if direction == 'F':
            self.msg = "Straight"

        cv2.putText(out_image, self.msg, org=(10, 240), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    # Display lane direction
                    fontScale=1, color=(255, 255, 255), thickness=2)
        if direction in 'LR':
            cv2.putText(out_image, curvature_msg, org=(
                # Display curvature
                10, 280), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.putText(
            out_image,
            "Vehicle is {:.2f} m away from center".format(pos),
            org=(10, 310),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.66,
            color=(255, 255, 255),
            thickness=2)  # Display distance from lane center
        return out_image  # Return the image with additional information

    # Method to measure lane curvature and vehicle position
    def measure_curvature(self):
        ym = 30/720  # Meters per pixel in y dimension
        xm = 3.7/700  # Meters per pixel in x dimension

        # Copy coefficients of polynomial fits
        left_fit = self.left_fit.copy()
        right_fit = self.right_fit.copy()

        # Compute y coordinate for evaluating curvature
        y_eval = 700 * ym

        # Compute curvature radius for left and right lanes
        left_curveR = (
            (1 + (2*left_fit[0] * y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        right_curveR = (
            (1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        # Compute vehicle position relative to lane center
        xl = np.dot(self.left_fit, [700**2, 700, 1])
        xr = np.dot(self.right_fit, [700**2, 700, 1])
        pos = (1280//2 - (xl+xr)//2)*xm

        # Return curvature radii and vehicle position
        return left_curveR, right_curveR, pos

    # Process input image and detect lane lines
    def forward(self, image):
        """Take a image and detect lane lines.

        Parameters:
            image (np.array): An binary image containing relevant pixels

        Returns:
            Image (np.array): An RGB image containing lane lines pixels and other details
        """
        # Extract features from the input image
        self.extract_features(image)
        # Fit polynomial curves to detected lane pixels
        return self.fit_poly(image)

## Xây dựng model để nhận diện làn đường

**Hàm 'FindLaneLines'** có chức năng xử lý ảnh đầu vào để tìm vạch lane trên đường.

1. **Khởi tạo:**
    - Khởi tạo một đối tượng 'CameraCalibration' để loại bỏ biến dạng ảnh từ camera.
    - Khởi tạo các đối tượng khác để thực hiện ngưỡng ảnh (Thresholding), biến đổi phối cảnh (PerspectiveTransformation) và tìm vạch lane (LaneLines).

2. **Xử lý ảnh 'forward':**
    - Nắn chỉnh ảnh đầu vào để loại bỏ biến dạng thấu kính (undistort) bằng 'CameraCalibration'.
    - Biến đổi phối cảnh ảnh để có góc nhìn từ trên xuống đường (perspective transform) bằng 'PerspectiveTransformation'.
    - Chuyển đổi ảnh đã biến đổi phối cảnh thành ảnh nhị phân để tách biệt vạch lane với phần còn lại (thresholding) bằng 'Thresholding'.
    - Tìm vạch lane trong ảnh nhị phân bằng 'LaneLines'.
    - Biến đổi phối cảnh ngược lại để đưa ảnh về phối cảnh ban đầu.
    - Trộn ảnh gốc với ảnh kết quả đã tìm thấy vạch lane để hiển thị kết quả.

3. **Xử lý ảnh thời gian thực 'process_image_rt':**
    - Gọi hàm 'forward' để xử lý ảnh theo các bước trên.

Nhìn chung, hàm 'FindLaneLines' sử dụng các chức năng xử lý ảnh khác nhau để tách vạch lane ra khỏi ảnh đường và hiển thị chúng lên ảnh gốc.

**Bài tập 4:** Hãy hoàn thành đoạn code sau bằng cách điền vào [...].                                  

In [ ]:
class FindLaneLines:
    def __init__(self):
        self.calibration = [...]
        self.thresholding = [...]
        self.transform = [...]
        self.lanelines = [...]

    # Process the input image and find lane lines
    def forward(self, image):
        # Create a copy of the input image
        output_image = np.copy([...])
        # Undistort the input image using camera calibration
        undistort_image = self.calibration.undistort([...])
        # Apply perspective transformation to bird-eye view image
        bird_eye_image = self.transform.forward([...])
        # Apply thresholding to the transformed image to extract lane pixels
        threshold_image = self.thresholding.forward([...])
        # Detect lane lines in the thresholded image
        laneline_image = self.lanelines.forward([...])
        # Reverse the perspective transformation to obtain the original perspective
        laneline_image = self.transform.backward([...])

        # Blend the processed image with the original image
        output_image = cv2.addWeighted(output_image, 1, laneline_image, 0.6, 0)
        # Plot additional information (lane direction, curvature, etc.) on the processed image
        output_image = self.lanelines.plot([...])
        # Return the final processed image
        return output_image

    # Process the input image and find lane lines in real-time
    def process_image_rt(self, image):
        # Forward the input image to the 'forward' method to find lane lines
        output_image = self.forward([...])
        # Return the processed image
        return output_image

In [42]:
class FindLaneLines:
    def __init__(self):
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    # Process the input image and find lane lines
    def forward(self, image):
        # Create a copy of the input image
        output_image = np.copy(image)
        # Undistort the input image using camera calibration
        undistort_image = self.calibration.undistort(image)
        # Apply perspective transformation to bird-eye view image
        bird_eye_image = self.transform.forward(undistort_image)
        # Apply thresholding to the transformed image to extract lane pixels
        threshold_image = self.thresholding.forward(bird_eye_image)
        # Detect lane lines in the thresholded image
        laneline_image = self.lanelines.forward(threshold_image)
        # Reverse the perspective transformation to obtain the original perspective
        laneline_image = self.transform.backward(laneline_image)

        # Blend the processed image with the original image
        output_image = cv2.addWeighted(output_image, 1, laneline_image, 0.6, 0)
        # Plot additional information (lane direction, curvature, etc.) on the processed image
        output_image = self.lanelines.plot(output_image)
        # Return the final processed image
        return output_image

    # Process the input image and find lane lines in real-time
    def process_image_rt(self, image):
        # Forward the input image to the 'forward' method to find lane lines
        output_image = self.forward(image)
        # Return the processed image
        return output_image

In [ ]:
class FindLaneLines:
    def __init__(self):
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    # Process the input image and find lane lines
    def forward(self, image):
        # Create a copy of the input image
        out_image = np.copy(image)
        # Undistort the input image using camera calibration
        image = self.calibration.undistort(image)
        # Apply perspective transformation to the undistorted image
        image = self.transform.forward(image)
        # Apply thresholding to the transformed image to extract lane pixels
        image = self.thresholding.forward(image)
        # Detect lane lines in the thresholded image
        image = self.lanelines.forward(image)
        # Reverse the perspective transformation to obtain the original perspective
        image = self.transform.backward(image)

        # Blend the processed image with the original image
        out_image = cv2.addWeighted(out_image, 1, image, 0.6, 0)
        # Plot additional information (lane direction, curvature, etc.) on the processed image
        out_image = self.lanelines.plot(out_image)
        # Return the final processed image
        return out_image

    # Process the input image and find lane lines in real-time
    def process_image_rt(self, image):
        # Forward the input image to the 'forward' method to find lane lines
        out_image = self.forward(image)
        # Return the processed image
        return out_image

## Khai báo các biến local

Chúng ta cần khai báo các biến local để phục vụ cho quá trình lập trình. Trong đó sẽ bao gồm:
- 'find_lane_lines' là một object thuộc class 'FindLaneLines()'
- 'url_image' là đường link để lấy hình ảnh từ cam.
- 'url_cmd' là đường link để truyền lệnh đến xe.
- 'speed_cmd' là tốc độ của xe.
- 'msg_to_dir_cmd' là định nghĩa các lệnh sang dạng số.


In [ ]:
find_lane_lines = FindLaneLines()
url_image = r'http://192.168.109.103/capture'
url_cmd = "http://192.168.109.103:81/message"
speed_cmd = 35
msg_to_dir_cmd = {
    "Straight": 1,
    "Left": 7,
    "Right": 5,
    "Stop": 9,
}

queue_image = Queue()

## Lấy dữ liệu hình ảnh

Hàm 'get_image()' có nhiệm vụ lấy hình ảnh liên tục từ một URL, chuyển đổi và xử lý chúng, sau đó đặt vào hàng đợi để các phần khác của chương trình có thể sử dụng các hình ảnh này.

In [ ]:
# Get image from server of the carkit
def get_image():
    while True:
        try:
            # Open the URL to fetch image
            image = urlopen(url_image, timeout=5)
            # Convert image data to NumPy array
            imagenp = np.asarray(bytearray(image.read()), dtype="uint8")
            image = cv2.imdecode(imagenp, -1)  # Decode the image
            image = cv2.resize(image, (1280, 720))  # Resize the image
            queue_image.put(image)  # Put the image into the queue
            # Check if the queue size exceeds the maximum limit
            if queue_image.qsize() >= 3:
                _ = queue_image.get()  # Pop the oldest image
        except Exception as e:
            print("Error fetching image:", e)
            time.sleep(1)  # Wait for 1 second before retrying

# Test lấy dữ liệu hình ảnh

Hàm 'display_image()' dùng để kiểm tra tính năng lấy ảnh từ bộ xe. Ảnh sẽ được lấy ra từ hàng đợi và hiển thị ra màn hình kèm với FPS.

In [ ]:
# Test the getting image process of carkit
def display_image():
    prev_time = time.time()  # Record the previous time
    while True:
        try:
            image = queue_image.get()  # Get an image from the queue
            image = find_lane_lines.process_image_rt(
                image)  # Process the image in real-time

            current_time = time.time()  # Record the current time
            fps = 1 / (current_time - prev_time)  # Calculate frames per second
            prev_time = current_time  # Update the previous time

            cv2.putText(image, f"FPS: {int(fps)}", (10, 30),
                        # Add FPS text to the image
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            cv2.imshow("Detected Image", image)  # Display the image

            if cv2.waitKey(1) & 0xFF == ord('q'):  # Check for 'q' key press to exit
                break
        except Exception as e:
            print("Error:", e)  # Print any errors

## Nhận diện làn đường

Hàm 'process_image()' được thiết kế liên tục xử lý hình ảnh từ hàng đợi, điều khiển phương tiện dựa trên kết quả xử lý và hiển thị hình ảnh đã xử lý cùng với FPS.

Quá trình này sẽ được thực hiện trong một vòng lặp vô hạn:
- Lấy hình ảnh từ hàng đợi.
- Đưa qua 'find_lane_lines.process_image_rt(img)' để xử lý hình ảnh và tìm các làn đường.
- Sau đó lấy thông điệp điều khiển xe từ 'find_lane_lines.lanelines.msg'.
- Chuyển đổi thông điệp điều khiển thành mã lệnh và gửi yêu cầu POST đến 'url_cmd' để điều khiển phương tiện.

Vòng lặp sẽ dừng lại khi ta nhấn phím 'q'.

Chức năng của hàm này là liên tục xử lý hình ảnh từ hàng đợi, điều khiển phương tiện dựa trên kết quả xử lý và hiển thị hình ảnh đã xử lý cùng với FPS.

In [ ]:
# Process image to extract line and motion control
def process_image():
    prev_time = time.time()  # Record the previous time
    while True:
        try:
            image = queue_image.get()  # Get an image from the queue
            current_time = time.time()  # Record the current time
            fps = 1 / (current_time - prev_time)  # Calculate frames per second
            prev_time = current_time  # Update the previous time
            # Process the image in real-time
            image = find_lane_lines.process_image_rt(image)
            msg = find_lane_lines.lanelines.msg  # Get the lane direction message
        except Exception as e:
            print("Error processing image:", e)  # Print errors
            cv2.putText(image, f"Not found", (150, 30),
                        # Add "Not found" text to the image
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            msg = "Stop"  # Set message to "Stop"

        # Get the command code corresponding to the message
        dir_cmd = msg_to_dir_cmd.get(msg, 9)
        data = {"Cmd": f"{dir_cmd} {speed_cmd}"}  # Construct the command data
        print(data)  # Print the command data
        # Send command to the vehicle
        response = requests.post(url_cmd, data=data)
        print(response.text)  # Print the response

        cv2.putText(image, f"FPS: {int(fps)}", (10, 30),
                    # Add FPS text to the image
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        cv2.imshow("Detected Image", image)  # Display the image

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Check for 'q' key press to exit
            msg = "Stop"  # Set default message to "Stop"
            # Get the command code corresponding to the message
            dir_cmd = msg_to_dir_cmd.get(msg, 9)
            # Construct the command data
            data = {"Cmd": f"{dir_cmd} {speed_cmd}"}
            # Send command to the vehicle
            response = requests.post(url_cmd, data=data)
            break  # Exit the loop

**Bài tập 5:** Hãy hoàn thành đoạn code sau bằng cách điền vào [...].                                  

Để tăng hiệu suất xử lí và giảm tải khối lượng công việc, chúng ta sẽ sử dụng kĩ thuật chạy đa luồng. Luồng một sẽ nhằm mục đích lấy hình ảnh từ server và lưu vào hàng đợi. Luồng hai sẽ xử lí ảnh từ hàng đợi và đưa lệnh cho xe.
- Luồng 1 lấy ảnh từ bộ ACE-kit
- Luồng 2 xử lý nhận diện làn đường 

In [ ]:
# Create two threads
thread1 = threading.Thread(target=[...], name='Thread 1')
thread2 = threading.Thread(target=[...], name='Thread 2')
# Start the threads
thread1.start()
thread2.start()
# Wait for the threads to finish
thread1.join()
thread2.join()
cv2.destroyAllWindows()

In [ ]:
# Create two threads
thread1 = threading.Thread(target=get_image, name='Thread 1')
thread2 = threading.Thread(target=process_image, name='Thread 2')
# Start the threads
thread1.start()
thread2.start()
# Wait for the threads to finish
thread1.join()
thread2.join()
cv2.destroyAllWindows()